# Get Tesla supercharger locations

#### Load Python tools and Jupyter config

In [2]:
import us
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data as vega_data
from tqdm.notebook import tqdm, trange
from random import randint
from time import sleep

In [3]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [4]:
place = "tesla"
place_formal = "Tesla superchargers"
color = "#cc0000"
today = pd.Timestamp.today().strftime("%Y-%m-%d")

---

## Scrape

#### Headers for the requests

In [17]:
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "priority": "u=1, i",
    "referer": "https://www.tesla.com",
    "sec-ch-ua": '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
}

params = {
    "translate": "en_US",
    "usetrt": "true",
}

response = requests.get(
    "https://www.tesla.com/cua-api/tesla-locations",
    params=params,
    headers=headers,
)

In [47]:
location_list = response.json()

In [48]:
src_df = pd.DataFrame(location_list)

In [55]:
src_df.sample(30)

,location_type,location_id,latitude,longitude,open_soon,title,address_line_1,address_line_2,city,province_state,postal_code,directions_link,country,_ignored,superchargerWinner
8201,[supercharger],409103,36.7940534,-75.9941442,1,locations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4495,"[vendor_collision, service, store]",alicantecarrerpluton,38.343131,-0.528219,0,locations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14960,[destination charger],dc535992,46.495754,11.868032,0,locations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6989,[supercharger],29909,45.0395597,9.7505216,1,locations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18368,[destination charger],54985,24.552644,-81.797189,0,locations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21074,[destination charger],429263,42.461844,-83.125983,0,locations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4986,[supercharger],DenverPAsupercharger,40.202912,-76.0159277,0,locations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
726,"[supercharger, nacs]",27472,37.362504,-122.026259,0,locations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2145,[service],12616,49.24475,-122.97334,1,locations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14197,[destination charger],52343,42.7563,143.05818,0,locations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Fetch the HTML content from the Tesla website
response = requests.get(
    "https://www.tesla.com/findus/list/superchargers/United+States", headers=headers
)
html_content = response.text
soup = BeautifulSoup(html_content, "html.parser")

In [6]:
vcards = soup.find_all("address", class_="vcard")

In [7]:
urls = []

for vcard in vcards:
    anchor = vcard.find("div", class_="anchor-container").find("a")
    if anchor:
        urls.append(anchor["href"])

In [8]:
# Base URL to concatenate with relative paths
base_url = "https://www.tesla.com"

In [9]:
# Example function to extract information from each location page
def extract_location_info(location_soup):
    # Extract Location Name and Street Address
    street_address_element = location_soup.find("span", class_="street-address")
    if street_address_element:
        street_address_parts = street_address_element.decode_contents().split("<br/>")
        location_name = street_address_parts[0].strip()
        street_address = (
            street_address_parts[1].strip() if len(street_address_parts) > 1 else ""
        )
    else:
        location_name = ""
        street_address = ""

    # Extract City, State, Zip Code
    locality = location_soup.find("span", class_="locality")
    if locality:
        locality_text = locality.get_text(strip=True)
        city_state_zip = locality_text.split(",")
        city = city_state_zip[0].strip()
        state_zip = (
            city_state_zip[1].strip().split(" ")
            if len(city_state_zip) > 1
            else ["", ""]
        )
        state = state_zip[0] if len(state_zip) > 0 else ""
        zip_code = state_zip[1] if len(state_zip) > 1 else ""
    else:
        city = ""
        state = ""
        zip_code = ""

    # Extract Coordinates from Driving Directions link
    directions_link = location_soup.find("a", string="Driving Directions")
    if directions_link and "daddr=" in directions_link["href"]:
        coordinates = directions_link["href"].split("daddr=")[1].split(",")
        latitude = coordinates[0].strip()
        longitude = coordinates[1].strip() if len(coordinates) > 1 else ""
    else:
        latitude = ""
        longitude = ""

    # Extract Charging Details
    charging_details = location_soup.find(string=lambda x: "Superchargers" in x)
    charging_details = charging_details.strip() if charging_details else ""

    # Extract Amenities
    amenities_elements = location_soup.select("ul.amenities-icons li a")
    amenities = [
        amenity.get("class")[0].replace("amenetie-icon-", "")
        for amenity in amenities_elements
    ]
    amenities = ", ".join(amenities)

    return {
        "Location Name": location_name,
        "Street Address": street_address,
        "City": city,
        "State": state,
        "Zip Code": zip_code,
        "Latitude": latitude,
        "Longitude": longitude,
        "Charging Details": charging_details,
        "Amenities": amenities,
    }

In [10]:
# List to store the collected information
data = []

# For each URL, get the page content and parse the details
for url in tqdm(urls, desc="Processing: "):
    full_url = base_url + url
    response = requests.get(full_url, headers=headers)
    location_soup = BeautifulSoup(response.content, "html.parser")

    # Extract information using the defined function
    location_info = extract_location_info(location_soup)
    data.append(location_info)
    sleep(randint(2, 5))

Processing:   0%|          | 0/1900 [00:00<?, ?it/s]

In [11]:
# Create DataFrame
df = pd.DataFrame(data)

In [13]:
df.to_json(
    f"data/processed/tesla_supercharger_locations_latest.json",
    indent=4,
    orient="records",
)

In [20]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [22]:
df["superchargers_count"] = (
    df["charging_details"]
    .str.split(",", expand=True)[0]
    .str.replace(" Superchargers", "")
)
df["when_available"] = (
    df["charging_details"].str.split(",", expand=True)[1].str.replace("available ", "")
)
df["power_capacity"] = (
    df["charging_details"].str.split(",", expand=True)[2].str.replace("up to ", "")
)

In [42]:
df = df.query('latitude !=""')

#### Create a mapping of state abbreviations to full state names using the us library

In [43]:
state_mapping = {state.abbr: state.name for state in us.states.STATES}

#### New column of full state names based on abbreviations

In [44]:
df["state_name"] = df["state"].map(state_mapping)

#### Make sure our brand name gets in the dataframe

In [45]:
df["brand"] = place_formal

#### Add fetch date

In [46]:
df["updated"] = today

---

## Geography

#### Make it a geodataframe

In [47]:
df_geo = df.copy()

In [48]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
).set_crs("4326")

---

## Maps

#### US states background

In [49]:
background = (
    alt.Chart(alt.topo_feature(vega_data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [50]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=5, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [51]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [52]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [53]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [54]:
gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)